<a href="https://colab.research.google.com/github/BlackCaveCatt/RPI-AI-TestComputerVision/blob/main/Copy_of_retrain_existing_model_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import some library code


*   `tensorflow` is the machine learning library we're using
*   `os` helps us interact with files and folders
*   `matplotlib` is for displaying charts and images
*   `numpy` helps us work with data to prepare it for `tensorflow` and review it afterwards


In [2]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np

This code gets the data you will use to train your model: pictures of cats and dogs.


In [3]:
import tensorflow_datasets as tfds
(raw_training, raw_validation, raw_testing), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.77P7KD_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.



This function takes an image and a label as inputs. The image is then converted into three sets of numbers representing the colours red, green, and blue for every pixel in the image. The combination of these colours can form any other colour. The red, green, and blue values are then converted from numbers between 0 and 255, to numbers between -1 and 1, as the model has been trained to work with values in that range. Finally, the image is resized based on the `IMAGE_SIZE` constant, to match the size the model was previously trained on. In this case, it's a 160 by 160 pixel square.


In [4]:
IMAGE_SIZE = 160

training_data = None

# Resize an image, and convert it into a form that tensorflow can read more easily
def prep_image(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
  return image, label

training_data = raw_training.map(prep_image)
validation_data = raw_validation.map(prep_image)
testing_data = raw_testing.map(prep_image)

These are versions of the functions from the previous project, so you can use them to test your model.

In [5]:
def get_image_from_url(image_url):
  # If the temporary test_image.jpg file already exists,
  # delete it so a new one can be made.
  if os.path.exists('/root/.keras/datasets/test_image.jpg'):
    os.remove('/root/.keras/datasets/test_image.jpg')

  image_path = tf.keras.utils.get_file('test_image.jpg', origin=image_url)
  return image_path

def print_predictions(predictions):
    for (prediction, number) in zip(predictions[0], range(1, len(predictions[0])+1)):
      print('{}. {} {:.2f}%'.format(number, prediction[1], prediction[2]*100))

def predict_with_old_model(image_url):
  image_path = get_image_from_url(image_url)

  image = tf.keras.preprocessing.image.load_img(image_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))

  plt.figure()
  plt.imshow(image)

  image = tf.keras.preprocessing.image.img_to_array(image)
  image = np.expand_dims(image, axis=0)

  prediction_result = original_model.predict(image, batch_size=1)
  predictions = tf.keras.applications.imagenet_utils.decode_predictions(prediction_result, top=15)

  print_predictions(predictions)

def predict_image(image_url):
  image_path = get_image_from_url(image_url)

  image = tf.keras.preprocessing.image.load_img(image_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))

  plt.figure()
  plt.imshow(image)

  image = tf.keras.preprocessing.image.img_to_array(image)
  image = np.expand_dims(image, axis=0)

  prediction_result = model.predict(image, batch_size=1)
  labels = metadata.features['label'].names
  print(labels[prediction_result.argmin()])

Import and test the MobileNetV2 model that you will retrain.



In [6]:
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

In the cell below, split your images into training, validation, and testing data.

In [7]:
original_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE, include_top=False)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Add the new layers to the model, to allow it to be retrained.

In [8]:
original_model.trainable = False

Set up your training epochs and train the new layers of the model.

In [9]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000
training_batches = training_data.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation_data.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
testing_batches = testing_data.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

Use the `predict_image` function to test your model.

In [10]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(2)
model = tf.keras.Sequential([
  original_model,
  global_average_layer,
  prediction_layer
])

In [11]:
# Add an input layer to define the shape
input_shape = (None, None, 3) # Assuming your images are RGB
input_layer = tf.keras.layers.Input(shape=input_shape)

model = tf.keras.Sequential([
  input_layer,  # Add this line
  original_model,
  global_average_layer,
  prediction_layer
])


In [12]:
BASE_LEARNING_RATE = 0.0001
# Use the learning_rate argument when instantiating the optimizer
optimizer = tf.keras.optimizers.RMSprop(learning_rate=BASE_LEARNING_RATE)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_160 (Functional)    │ (None, None, None, 1280)    │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2)                   │           2,562 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,260,546 (8.62 MB)

 Trainable params: 2,562 (10.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [13]:
TRAINING_EPOCHS = 10

In [14]:
with tf.device('/device:GPU:0'):
  history = model.fit(training_batches,
                      epochs=TRAINING_EPOCHS,
                      validation_data=validation_batches)

Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 64s 89ms/step - accuracy: 0.5756 - loss: 1.5500 - val_accuracy: 0.6066 - val_loss: 0.6960
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 66s 73ms/step - accuracy: 0.6197 - loss: 0.6932 - val_accuracy: 0.6131 - val_loss: 0.6961
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 44s 72ms/step - accuracy: 0.6198 - loss: 0.6905 - val_accuracy: 0.6144 - val_loss: 0.6961
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 82s 73ms/step - accuracy: 0.6172 - loss: 0.6907 - val_accuracy: 0.6122 - val_loss: 0.6960
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 44s 72ms/step - accuracy: 0.6146 - loss: 0.6906 - val_accuracy: 0.6148 - val_loss: 0.6960
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 82s 73ms/step - accuracy: 0.6180 - loss: 0.6907 - val_accuracy: 0.6178 - val_loss: 0.6961
Epoch 7/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 82s 72ms/step - accuracy: 0.6250 - loss: 0.6907 - val_accuracy: 0.6182 - val_loss: 0.6961
Epoch 8/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 43s 72ms/step - accuracy: 0.6247 - loss: 0.6902 - 